In [54]:
import re
import gensim
import numpy as np
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from gensim import corpora, models
from pprint import pprint
from collections import Counter
from collections import defaultdict

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
import os
os.chdir("..")
from question_generation.pipelines import pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tanve\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stemmer = SnowballStemmer('english')
def lemmatize_stemming(text):
    '''Lemmatizes and stems input word'''
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    """Lemmatizes and stems all input texts"""
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [4]:
with open("data/econ3_8.txt") as f:
    doc1 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [5]:
with open("data/econ3_7.txt") as f:
    doc2 = f.read().strip()
#words = doc.split(' ')
#processed_doc = preprocess(doc)

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_doc)

In [ ]:
doc.split(".")

In [ ]:
sentences = []
curr = None
for sent in doc.split("."):
    if re.search(r"([:])", sent):
        if curr is not None:
            sentences.append(curr)
        curr = sent
    else:
        curr += "."+sent
sentences

In [17]:
# TODO filter meaningless sentences
def get_teacher_text(doc):
    "Extracts text spoken by teacher"
    matches = re.findall(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)
    splits = re.split(r"(?:(?:[A-Z][a-z]*\s)*(?:[A-Z][a-z]*)):", doc)[1:]
    teacher = Counter(matches).most_common(1)[0][0]
    return ''.join([splits[i] for i in range(len(splits)) if matches[i] == teacher])
def get_sents(doc):
    return [sent for sent in sent_tokenize(doc)]
def get_filtered_sents(doc):
    return [sent for sent in sent_tokenize(doc) if len(re.findall(r"\s", sent)) > 10]

In [22]:
sents = get_sents(get_teacher_text(doc1))
filtered_sents = get_filtered_sents(get_teacher_text(doc1))

In [47]:
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]

data_words = list(sent_to_words(filtered_sents))
# remove stop words
data_words = remove_stopwords(data_words)

id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

num_topics = 5

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
lda_model

In [48]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.046*"growth" + 0.034*"okay" + 0.020*"rate" + 0.019*"know" + 0.018*"going" '
  '+ 0.012*"one" + 0.011*"average" + 0.011*"really" + 0.011*"per" + '
  '0.010*"gdp"'),
 (1,
  '0.028*"one" + 0.024*"growth" + 0.016*"know" + 0.016*"going" + '
  '0.015*"average" + 0.014*"right" + 0.013*"years" + 0.012*"take" + '
  '0.011*"rate" + 0.010*"would"'),
 (2,
  '0.023*"growth" + 0.017*"let" + 0.013*"gdp" + 0.013*"per" + 0.012*"capita" + '
  '0.011*"see" + 0.010*"living" + 0.010*"average" + 0.009*"rate" + '
  '0.009*"things"'),
 (3,
  '0.018*"per" + 0.017*"gdp" + 0.016*"year" + 0.014*"one" + 0.012*"capita" + '
  '0.011*"average" + 0.011*"years" + 0.010*"going" + 0.009*"take" + '
  '0.009*"see"'),
 (4,
  '0.019*"per" + 0.018*"much" + 0.013*"gdp" + 0.012*"get" + 0.011*"know" + '
  '0.011*"right" + 0.011*"one" + 0.011*"time" + 0.011*"china" + '
  '0.010*"capita"')]


In [89]:
clusters = defaultdict(str)
labels = lda_model.inference(corpus)[0].argmax(axis=1)
for i in range(len(filtered_sents)):
    clusters[labels[i]] += filtered_sents[i] + " "
for topic in clusters:
    clusters[topic] = re.sub(r"\s+", " ", clusters[topic])
    clusters[topic] = re.sub(r"[',]", "", clusters[topic])
clusters

defaultdict(str,
            {0: 'Looks like I feel Im only hiding my tiredness behind my makeup. I said oh if I didnt have that conference I need to go to later in Catholic I would have canceled today. Okay so so it might be just because what we were just fielding all these emails this issue and that. Obviously the essay questions have been answered but the the Mean was not as high as usual we think its just because of all the stuff going on so that it no promises but that likely is likely to meet a Generous curb. I think pertaining to this but hopefully you figured out you didnt need to know that for the midterm. Okay so most macro series grow at what we call compound rates. Now closely related concepts are what are called geometric growth an exponential growth. In fact were going to use some approximations from exponential growth to talk about compound growth. So if the growth rate is 4% and you write it as a equals point 04 Okay so in year one. In fact We can say in general if we w

In [ ]:
for topic in clusters:
    print(topic)
    for sent in get_sents(clusters[topic]):
        try:
            print(nlp(sent))
        except:
            print(sent)
            continue
    print()

0
[{'answer': 'hiding my tiredness behind my makeup', 'question': "What do I feel like I'm only hiding behind my makeup?"}]
[{'answer': 'canceled', 'question': 'What would happen if I didnt have a conference in Catholic?'}]
[{'answer': 'fielding all these emails', 'question': 'What was the reason for this issue?'}]


In [96]:
nlp("Supposed to a is equal to be time see times d. And at each series say is growing the time although it could also be decreasing.")

ValueError: substring not found

In [27]:
doc_lda

In [6]:
nlp = pipeline("multitask-qa-qg")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [21]:
nlp("Right recall from Chapter 16 that average labor product ALP is equal to total output divided by the number of workers or if you want to count up hours per worker, it could be total output divided by the number of hours worked.")

[{'answer': 'the number of hours worked',
  'question': 'What is the total output divided by?'}]